<a href="https://colab.research.google.com/github/tolom131/Human-Activity-Recognition/blob/main/Tensorflow/transformer_autoencoder_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
import numpy as np

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, LSTM, Conv1D, Activation, MaxPooling1D, UpSampling1D, concatenate

from tensorflow.keras import layers

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
from tensorflow.keras import backend as K

import random as rn
from resource import *
import time
import math
import sys
from collections import Counter

In [2]:
from google.colab import drive 
drive.mount('/content/drive/', force_remount=True)

directory_data  = './drive/MyDrive/HAR/'
filename_data   = 'WISDM_ar_v1.1_raw.txt'
 
sys.path.append('/content/drive/MyDrive/HAR/')
import wisdm_1_1
import wisdm_2_0
import UCI_dataset
# original_x, original_y, num_classes = wisdm_1_1.create_wisdm_1_1(directory_data + filename_data)
# original_x, original_y, num_classes = wisdm_2_0.create_wisdm_2_0(directory_data + filename_data)
original_x, original_y, x_test, y_test = UCI_dataset.UCI_dataset(directory_data)

Mounted at /content/drive/


In [3]:
from sklearn.utils import shuffle
def shift(x, y):
    new = []
    for i in range(3):
        temp = np.roll(x, i*65, axis=1)
        new.extend(temp)
    x = np.array(new)
    y = np.concatenate([y]*3)
    x, y = shuffle(x, y, random_state=42)
    return x,y

from scipy.interpolate import CubicSpline
def GenerateRandomCurves(X, sigma=0.2, knot=4):
    xx = (np.ones((X.shape[1], 1))*(np.arange(0, X.shape[0], (X.shape[0]-1)/(knot+1)))).transpose()
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, X.shape[1]))
    x_range = np.arange(X.shape[0])

    cs_x = CubicSpline(xx[:, 0], yy[:, 0])
    cs_y = CubicSpline(xx[:, 1], yy[:, 1])
    cs_z = CubicSpline(xx[:, 2], yy[:, 2])
    return np.array([cs_x(x_range), cs_y(x_range), cs_z(x_range)]).transpose()

def DistortTimesteps(X, sigma=0.2):
    tt = GenerateRandomCurves(X, sigma)
    tt_cum = np.cumsum(tt, axis=0)

    t_scale = [(X.shape[0]-1)/tt_cum[-1, 0], (X.shape[0]-1)/tt_cum[-1, 1], (X.shape[0]-1)/tt_cum[-1, 2]]
    tt_cum[:, 0] = tt_cum[:, 0] * t_scale[0]
    tt_cum[:, 1] = tt_cum[:, 1] * t_scale[1]
    tt_cum[:, 2] = tt_cum[:, 2] * t_scale[2]

    return tt_cum

def DA_TimeWarp(X, sigma=0.2):
    tt_new = DistortTimesteps(X, sigma)
    X_new = np.zeros(X.shape)
    x_range = np.arange(X.shape[0])

    X_new[:, 0] = np.interp(x_range, tt_new[:, 0], X[:, 0])
    X_new[:, 1] = np.interp(x_range, tt_new[:, 1], X[:, 1])
    X_new[:, 2] = np.interp(x_range, tt_new[:, 2], X[:, 2])

    return X_new

def TimeWarp(X, Y):
    for i in range(X.shape[0]):
        data = X[i, :, :]
        if i == 0:
            trans_list = DA_TimeWarp(data).reshape(-1, X.shape[1], X.shape[2])
        else:
            trans = DA_TimeWarp(data).reshape(-1, X.shape[1], X.shape[2])
            trans_list = np.concatenate([trans_list, trans], axis=0)

    return trans_list, Y

In [4]:
def ConvolutionalBlock(dim_shape, filters=128, is_encoder=True):
    initializer = initializers.GlorotNormal()

    # is_encoder : "encoder" means that layers is part of encoder, else part of decoder
    if is_encoder:
        out_filter = filters * 2
    else:
        out_filters = int(filters / 2)

    inputs = Input(shape=dim_shape)
    x = Conv1D(filters=filters, kernel_size=3, padding="same", kernel_initializer=initializer)(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    if is_encoder:
        x = Conv1D(filters=filters * 2, kernel_size=3, strides=2, padding="same", kernel_initializer=initializer)(x)
        outputs = Activation("relu")(x)
    else:
        x = Conv1D(filters=filters // 2, kernel_size=3, strides=1, padding="same", kernel_initializer=initializer)(x)
        x = Activation("relu")(x)
        outputs = UpSampling1D()(x)

    model = Model(inputs, outputs)
    return model

def MultiheadBlock(dim_shape, filters=128):
    initializer = initializers.GlorotNormal()

    inputs = keras.Input(shape=dim_shape)
    x = layers.MultiHeadAttention(key_dim=512, num_heads=8)(inputs, inputs)
    x = x + inputs
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = Conv1D(filters, kernel_size=3, padding="same", kernel_initializer=initializer)(x)
    outputs = Activation("relu")(x)

    model = Model(inputs, outputs)
    return model

def Transformer_Autoencoder(x_train, y_train, x_val, y_val, alpha=0.1, shape=(200, 3)):

    ########################################################################
    # functions
    ########################################################################
    earlystop = EarlyStopping(patience=100, monitor='val_loss', mode='min', restore_best_weights=True)
    callbacks_list = [earlystop]
    initializer = initializers.GlorotNormal()
    adam = Adam(learning_rate=0.01)
    num_classes = 6

    ########################################################################
    # encoder
    ########################################################################
    inputs = Input(shape=shape)
    x = ConvolutionalBlock(shape, filters=64, is_encoder=True)(inputs)
    x = MultiheadBlock((shape[0]//2, 128), 128)(x)
    x = ConvolutionalBlock((shape[0]//2, 128), filters=128, is_encoder=True)(x)
    x = MultiheadBlock((shape[0]//4, 256), 256)(x)
    encoded = ConvolutionalBlock((shape[0]//4, 256), filters=256, is_encoder=True)(x)

    ########################################################################
    # decoder
    ######################################################################## 
    x = ConvolutionalBlock((shape[0]//8, 512), 512, is_encoder=False)(encoded)
    x = MultiheadBlock((shape[0]//4, 256), 256)(x)
    x = ConvolutionalBlock((shape[0]//4, 256), filters=256, is_encoder=False)(x)
    x = MultiheadBlock((shape[0]//2, 128), 128)(x)
    x = ConvolutionalBlock((shape[0]//2, 128), filters=128, is_encoder=False)(x)
    decoded = Conv1D(shape[1], kernel_size=3, padding="same", name="decoded", kernel_initializer=initializer)(x)

    ########################################################################
    # classifier
    ########################################################################
    x = Conv1D(filters=64, kernel_size=3, input_shape=(shape[0]//8, 512), kernel_initializer=initializer)(encoded)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv1D(filters=128, kernel_size=3, kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = LSTM(128, kernel_initializer=initializer)(x)
    classified = Dense(num_classes, activation="softmax", name="classified", kernel_initializer=initializer)(x)

    ########################################################################
    # model
    ########################################################################
    model = Model(inputs=[inputs], outputs=[classified, decoded])
    model.compile(optimizer=adam, loss=["categorical_crossentropy", "mse"], loss_weights=[alpha, 1-alpha], metrics=["accuracy"])
    history = model.fit([x_train], [y_train, x_train], validation_data = ([x_val], [y_val, x_val]), batch_size = 64, epochs=600, verbose=1)
    return history, model

In [5]:
# 80%, 10%, 10%로 set 나누기
x_train, x_val, y_train, y_val = train_test_split(original_x, original_y, test_size=0.1, stratify=original_y)
# x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5, stratify=y_val)

x_timewarp, y_timewarp  = TimeWarp(x_train, y_train)
x_train,    y_train     = shift(x_train, y_train)
x_train = np.concatenate([x_timewarp, x_train], axis=0)
y_train = np.concatenate([y_timewarp, y_train], axis=0)
print("shape of x_train : ", x_train.shape)

shape of x_train :  (26464, 128, 6)


In [6]:
history, model = Transformer_Autoencoder(x_train, y_train, x_val, y_val, alpha=0.5, shape=x_train.shape[1:])

Epoch 1/600
414/414 [==============================] - 52s 107ms/step - loss: 0.6046 - classified_loss: 1.1236 - decoded_loss: 0.0856 - classified_accuracy: 0.4196 - decoded_accuracy: 0.1834 - val_loss: 0.6276 - val_classified_loss: 1.1606 - val_decoded_loss: 0.0946 - val_classified_accuracy: 0.4334 - val_decoded_accuracy: 0.0000e+00
Epoch 2/600
414/414 [==============================] - 43s 104ms/step - loss: 0.5601 - classified_loss: 1.0476 - decoded_loss: 0.0726 - classified_accuracy: 0.4575 - decoded_accuracy: 0.1563 - val_loss: 1.9052 - val_classified_loss: 3.7157 - val_decoded_loss: 0.0946 - val_classified_accuracy: 0.1345 - val_decoded_accuracy: 0.2151
Epoch 3/600
414/414 [==============================] - 43s 103ms/step - loss: 0.5873 - classified_loss: 1.1019 - decoded_loss: 0.0726 - classified_accuracy: 0.4189 - decoded_accuracy: 0.1649 - val_loss: 1.0405 - val_classified_loss: 1.9863 - val_decoded_loss: 0.0947 - val_classified_accuracy: 0.1984 - val_decoded_accuracy: 0.2151


In [7]:
test_results = model.evaluate(x_test, y_test)

# print("maximum train acc : ", max(history.history["accuracy"]))
# print("maximum valid acc : ", max(history.history["val_accuracy"]))
print("test acc : ", test_results[1])
print("test loss : ", test_results[0])


y_pred = model.predict([x_test])
score = f1_score(y_test.argmax(axis=1), y_pred.argmax(axis=1), average="macro")
print("f1 score : ", score)
matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(matrix)

label = ["Jogging", "LyingDown", "Sitting", "Stairs", "Stading", "Walking"]
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(matrix, interpolation="nearest")
fig.colorbar(cax)
ax.set_xticklabels(['']+label)
ax.set_yticklabels(['']+label)
plt.show()

InvalidArgumentError: ignored